In [32]:
# ------------------------------------------------------
# Letter State Machine
# ------------------------------------------------------
LETTERS = "ETIANMSURWDKGOHVF L PJBXCYZQ"

CURRENT_LETTER = 0

'''
Initializes the machine
'''
def init_letters():
    global CURRENT_LETTER

    CURRENT_LETTER = 0

'''
Shifts the state machine to the next letter

Params:
- is_dash(bool): is really some bit indiciating if the incoming symbol is a dot or dash
'''
def shift_letter(is_dash: bool) -> None:
    global CURRENT_LETTER

    CURRENT_LETTER = CURRENT_LETTER * 2 + is_dash + 1

'''
Gets the letter that the current state machine is on

Returns:
The length 1 string that has the character
None, if CURRENT_LETTER is not in the range of the list of letters
'''
def get_letter() -> str: 
    global LETTERS, CURRENT_LETTER

    ret_letter = ""

    if CURRENT_LETTER > 0 and CURRENT_LETTER <= len(LETTERS):
        ret_letter = LETTERS[CURRENT_LETTER - 1]

    CURRENT_LETTER = 0

    return ret_letter

'''
Finishes the machine and grabs the last letter
'''
def finalize() -> str:
    return get_letter()

In [33]:
from enum import IntEnum

BEAT_ERROR_RANGE = 0.15

'''
A list of the possible parsings from the morse code
'''
class MEANING(IntEnum):
    # Symbol
    DOT = 0
    DASH = 1

    # Pauses
    NEXT_SYMBOL = 2
    NEXT_LETTER = 3
    NEXT_WORD = 4

    # Unknown
    UNKNOWN = -1
        
'''
Removes noise from the beat_duration
'''
def remove_noise(beat_nums:float) -> int:
    global BEAT_ERROR_RANGE

    error_ranges = [(dur, dur * (1 - BEAT_ERROR_RANGE), dur * (1 + (BEAT_ERROR_RANGE if dur != 7 else 1))) for dur in (1,3,7)]

    for (duration, duration_min, duration_max) in error_ranges:
        if beat_nums >= duration_min and beat_nums <= duration_max:
            return duration

    return -1

In [34]:
'''
Processes some parsed information and interacts with the "letter" state machine

Params:
- meaning (MEANING): some parsed info from the bitstream input

Returns:
Either a letter, a letter + a space, or None
'''
def process(meaning: MEANING) -> str | None:
    ret_proc = None
    match meaning:
        case MEANING.DOT:
            shift_letter(MEANING.DOT)
        case MEANING.DASH:
            shift_letter(MEANING.DASH)
        case MEANING.NEXT_LETTER:
            ret_proc = get_letter()
        case MEANING.NEXT_WORD:
            ret_proc = get_letter() + " "

    return ret_proc

In [35]:
# ------------------------------------------------------
# Beat Decoder
# ------------------------------------------------------
BEAT_DURATION = 1 # How many bits is a beat?

PREV_BIT = 0 # The value of the previous bit
NUM_OF_BIT = 0 # The number of occurrences of "prev bit"

'''
Determines what the last string of bit B means

Returns:
The information as one of the discrete meanings above.
'''
def parse_prev_inputs():
    global PREV_BIT, NUM_OF_BIT

    # TODO: eventually make this into some type of range
    beats = remove_noise(NUM_OF_BIT / BEAT_DURATION)

    meaning = None
    match (PREV_BIT, beats):
        case (1, 1):
            meaning = MEANING.DOT
        case (1, 3):
            meaning = MEANING.DASH
        case (0, 1):
            meaning = MEANING.NEXT_SYMBOL
        case (0, 3):
            meaning = MEANING.NEXT_LETTER
        case (0, 7):
            meaning = MEANING.NEXT_WORD
        case _:
            meaning = MEANING.UNKNOWN
    return meaning

'''
Processes the next bit

Params:
- bit(int): either a 0 or 1 that is either an "on" or "off" signal from the sender

Returns:
A letter if we have reached the end of a letter/word, else None
'''
def process_next_bit(bit: int) -> str | None:
    global PREV_BIT, NUM_OF_BIT

    if bit == PREV_BIT:
        NUM_OF_BIT += 1
    else:
        meaning = parse_prev_inputs()
        maybe_output = process(meaning)

        PREV_BIT = bit
        NUM_OF_BIT = 1

        return maybe_output

In [36]:
import sys

In [37]:
# ------------------------------------------------------
# Translation
# ------------------------------------------------------
def translate(input_file, out = None):
    if out is None:
        stdout = sys.stdout
    else:
        stdout = open(out, "w")

    with open(input_file, "r") as in_file:
        init_letters()
        for line in in_file:
            for bit in line:
                if bit in [" ", "\n"]:
                    continue

                bit = int(bit)
                
                letter = process_next_bit(bit)

                if letter is not None:
                    print(letter, end = "", file = stdout)
                    
        print(finalize(), file = stdout)

    if out is not None:
        stdout.close()
        

In [38]:
from pathlib import Path

In [39]:
'''
Basic Test Template to make other tests out of
'''
def translate_test(in_file, golden_file, result_file = None):
    if result_file is None:
        dot_split = in_file.split(".")
        result_file = "{}_out.{}".format("".join(dot_split[:-1]), dot_split[-1])
    
    translate(in_file, out=result_file)

    correct_output = True

    with open(result_file, "r") as results:
        with open(golden_file, "r") as golden:
            next_result = results.readline().strip().lower()
            next_golden = golden.readline().strip().lower()

            while next_result != "" and next_golden != "":
                if next_result != next_golden:
                    print(f"'{next_result}' does not match '{next_golden}'")
                    correct_output = False
                    break
                next_result = results.readline()
                next_golden = golden.readline()

            if next_result != next_golden:
                print(f"One was empty:\n'{next_result}' does not match '{next_golden}'")
                correct_output = False
            
    Path(result_file).unlink()

    return correct_output
                 


In [45]:
'''
Tests if the alphabet with no spaces is accurate
'''
def test1():
    translate_test("decoder_tests/test1.txt", "decoder_tests/golden_out1.txt")
    
test1()

In [42]:
'''
Tests if the alphabet with 1 space is accuate
'''
def test2():
    translate_test("decoder_tests/test2.txt", "decoder_tests/golden_out2_3.txt")

test2()

In [43]:
'''
Tests if multiple spaces are ommitted
'''
def test3():
    translate_test("decoder_tests/test3.txt", "decoder_tests/golden_out2_3.txt")

test3()

In [44]:
'''
Tests the "the quick brown fox jumps over the lazy dog"
'''
def test4():
    translate_test("decoder_tests/test4.txt", "decoder_tests/golden_out4.txt")